This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

In [24]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
small_params = dict(start_date='2020-07-20', end_date='2020-07-20', api_key=API_KEY)
small_res = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=small_params)

In [25]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
small_json = small_res.json()
small_json.keys()

dict_keys(['dataset_data'])

In [26]:
#inspect the format of the data
small_json['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2020-07-20',
 'end_date': '2020-07-20',
 'frequency': 'daily',
 'data': [['2020-07-20',
   88.25,
   89.5,
   87.55,
   88.95,
   None,
   227635.0,
   20134893.65,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [31]:
params = dict(start_date='2017-01-01', end_date='2017-12-31', api_key=API_KEY)
res = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=params)
json_dict = dict(res.json()['dataset_data'])

In [140]:
#extract the desired information from the json data
open_data = dict()
change_data = dict()
closing_data = []
volume_list = []
for (date, open_price, high_price, low_price, close_price, change_price, traded_volume, turnover, last_price, 
     daily_traded_units, daily_turnover) in json_dict['data']:
    if open_price!=None:
        open_data[date] = open_price
    if (high_price!=None) & (low_price!=None):
        change_data[date] = high_price - low_price
    closing_data.append((date, close_price))
    volume_list.append(traded_volume)

In [141]:
#find the highest and lowest opening price:
max_open = max(open_data, key=open_data.get)
min_open = min(open_data, key=open_data.get)
print("The highest opening price was", open_data[max_open], "on", max_open)
print("The lowest opening price was", open_data[min_open], "on", min_open)

The highest opening price was 53.11 on 2017-12-14
The lowest opening price was 34.0 on 2017-01-24


In [142]:
#the largest change in any one day (based on High and Low price)
max_change = max(change_data, key=change_data.get)
print("The largest change in one day was", change_data[max_change], "on", max_change)

The largest change in one day was 2.8100000000000023 on 2017-05-11


In [143]:
#the largest change between any two days (based on Closing Price)
#sort the list by date
closing_data.sort()
#make a new list of difference in price
closing_data_change = [abs(closing_data[i+1][1] - closing_data[i][1]) for i in range(len(closing_data)-1)]
max(closing_data_change)

2.559999999999995

In [144]:
#average daily trading volume during this year
avg_vol = sum(volume_list) / len(volume_list)
print(avg_vol)

89124.33725490196


In [145]:
#median trading volume during this year
def med_vol(vol_list):
    vol_list.sort()
    if len(vol_list) % 2 == 1:
        middle = int((len(vol_list) - 1) / 2)
        return(vol_list[middle])
    else:
        mid_upper = int(len(vol_list) / 2)
        mid_lower = mid_upper - 1
        med = (vol_list[mid_upper] + vol_list[mid_lower]) / 2
        return(med)
med_vol(volume_list)

76286.0